In [541]:
import scipy

import numpy as np

import pandas as pd
from pandas import Series, DataFrame

from sklearn import preprocessing, metrics
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV
from scipy.stats.distributions import randint

import csv


import plotly.plotly as py
# import visplots
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
import plotly.graph_objs as go
from plotly import tools
import plotly
plotly.tools.set_credentials_file(username='ra123', api_key='JtjRYEowQy5bMB6Z9lUD')

from nltk import word_tokenize, wordpunct_tokenize
from nltk.tokenize import wordpunct_tokenize
from nltk.probability import FreqDist, DictionaryProbDist, ELEProbDist, sum_logs
from nltk.classify.api import ClassifierI

# getting around the ascii characters
from django.utils.encoding import smart_str, smart_unicode

In [542]:
# Import the clean tweets (text and date, then through machine learning)
twitter_raw = pd.read_csv("C:\\Users\\Student24\\Documents\\twitter\\plotly\\Clean_Tweets1.csv", sep=',', delimiter=None)

# Delete tweets containing key words

In [543]:
twitter_cleaned=twitter_raw[twitter_raw['text'].str.lower().str.contains \
                   ("great smog of london|spanish|sex|porn|anal|pov|bbw|milf|sexy|shemale|sexyfishrestaurant|\
                   nude|sluts|super hot blonde|adult video|erotic|18+|dirty fun|killer fog|rihanna")==False]

twitter_cleaned = twitter_cleaned.reset_index(drop=True)

twitter_cleaned['text'].count()

8940

#  Machine Learning to clean tweets

# 1. Create the Maching learning

In [544]:
twlist = []

with open(r"C:\\Users\\Student24\\Documents\\twitter\\plotly\\trained_weather_NoRT.csv", "r") as t:
    tweets_raw = pd.read_csv(t)

tweets = tweets_raw[['text', 'weather']].values.tolist()

twlist = [tuple(l) for l in tweets] # turn nested list of lists into list of tuples
twtokens = []

In [545]:
for (words, weather) in twlist:
#    words_filtered = [e.lower().decode('utf8') for e in words.split() if len(e) >= 3 and len(e) <= 10] # and <= 10
    words_filtered = [unicode(e.lower(), errors = 'replace') for e in words.split() if len(e) >= 3 and len(e) <= 10] # and <= 10
    twtokens.append((words_filtered, weather))

In [546]:
import nltk
def get_words_in_tweets(tweets):
    all_words = []
    for (words, weather) in tweets:
        all_words.extend(words)
    return all_words

def get_word_features(wordlist):
    wordlist = nltk.FreqDist(wordlist)
    word_features = wordlist.keys()
    return word_features

In [547]:
word_features = get_word_features(get_words_in_tweets(twtokens))

In [548]:
# The Classifier
def extract_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in document_words)
    return features

In [549]:
training_set = nltk.classify.apply_features(extract_features, tweets)

# train the classifier
classifier = nltk.NaiveBayesClassifier.train(training_set)

print classifier.show_most_informative_features(32)

Most Informative Features
    contains(summer���s) = False              no : yes    =      1.0 : 1.0
           contains(66%) = False              no : yes    =      1.0 : 1.0
      contains(forecast) = False              no : yes    =      1.0 : 1.0
     contains(@thedoors) = False              no : yes    =      1.0 : 1.0
      contains(@pc24fym) = False              no : yes    =      1.0 : 1.0
    contains(infinit...) = False              no : yes    =      1.0 : 1.0
       contains(finally) = False              no : yes    =      1.0 : 1.0
        contains(1012mb) = False              no : yes    =      1.0 : 1.0
       contains(provide) = False              no : yes    =      1.0 : 1.0
    contains(@moosawi17) = False              no : yes    =      1.0 : 1.0
      contains(mentally) = False              no : yes    =      1.0 : 1.0
          contains(#now) = False              no : yes    =      1.0 : 1.0
         contains(rainy) = False              no : yes    =      1.0 : 1.0

# 2. Classify London Tweets

In [550]:
df = pd.DataFrame(twitter_cleaned)

tx = df['text']
df['text'] = tx

In [551]:
dffinal = df[['text']]
dffinal['date'] = df['Date']

In [552]:
for i in range(len(df.index)):
    if classifier.classify(extract_features((smart_str(df['text'][i])).split())) == 'yes':
        dffinal['text'][i] = smart_str(df['text'][i])
    else:
        dffinal['text'][i] = None

count = 0
for i in range(len(df.index)):
    if dffinal['text'][i] != None:
        count += 1

count2 = 0
dffinaltrained = pd.DataFrame({'date' : pd.Series(range(count), index=range(count)), 'text' : pd.Series(range(count), index=range(count))})
for i in range(len(df.index)):
    if dffinal['text'][i] != None:
        dffinaltrained['text'][count2] = dffinal['text'][i]
        dffinaltrained['date'][count2] = dffinal['date'][i]
        count2 += 1

C:\Users\Student24\Anaconda2\lib\site-packages\ipykernel\__main__.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [553]:
dffinaltrained['text'].count()

8093

# Define function which will count the weather words

In [554]:
def wordCount(tweet):
    # List of words we are looking at
    weather_words = ['breeze', 'breezy', 'cloudy', 'cold', 'ice', 'icy', 'icey', 'drizzle', 'frost', 'wind', 'mild', 'dew', 
                     'freezing', 'downpour', 'shower', 'rain', 'frost', 'nippy', 'hail', 'temperature', 'gail', 'gust',
                     'sleet', 'heat', 'storm', 'slush', 'fog', 'foggy', 'flood', 'visibility', 'warm', 'warmer',
                     'mist', 'frosty', 'misty', 'chilly', 'thunder', 'lightning', 'snow', 'snowing', 'hot', 'sun', 'sunny',
                     'boiling', 'baltic', 'burn']
    # Create a new dictionnary
    counts = dict()
    for t in tweet:
        if t in weather_words:
            counts[t] = counts.get(t,0) + 1

    return counts

# def df_count

In [555]:
# Check the days and number of tweets per day
#dffinal['Date']=(pd.to_datetime(df['created_at'])).dt.date
dffinaltrained.groupby(['date']).count()


,text
date,
2016-12-14,300
2016-12-15,255
2016-12-16,795
2016-12-19,791
2016-12-20,804
2016-12-21,421
2016-12-22,463
2016-12-23,438
2016-12-24,447


In [556]:
# Create a new dataframe with the date
tw_date = dffinaltrained.groupby(['date']).count()
tw_date.index

Index([u'2016-12-14', u'2016-12-15', u'2016-12-16', u'2016-12-19',
       u'2016-12-20', u'2016-12-21', u'2016-12-22', u'2016-12-23',
       u'2016-12-24', u'2016-12-25', u'2016-12-26', u'2016-12-27',
       u'2016-12-28', u'2016-12-30', u'2016-12-31', u'2017-01-01',
       u'2017-01-02'],
      dtype='object', name=u'date')

In [557]:
# Creates dataframe of total tweets for each day
df_count = dffinaltrained.groupby(['date']).count()

In [558]:
# Resets index
df_count.reset_index(level = 0, inplace = True)

In [559]:
df_count.rename(columns = {'text':'total_count'}, inplace = True)

In [560]:
df_count.head()

,date,total_count
0,2016-12-14,300
1,2016-12-15,255
2,2016-12-16,795
3,2016-12-19,791
4,2016-12-20,804


# Going through all tweets then count words and put them in final DataFrame (per day) for all words

In [561]:
tw_final_all = DataFrame() #makes tw_final_all a data frame
tw_final_all['date'] = tw_date.index
d2_list=[]
d2_len=[]
tw_final_all['Keywords'] = None

In [562]:
for i in range(len(tw_date.index)):
    df = dffinaltrained[(dffinaltrained['date'] == tw_final_all['date'][i])]
    twitterlist = []
    for t in range(len(df.index)):
        twitterlist.append(dffinaltrained['text'][t].lower().split())

    # Flatten the list so all values in the same list
    tweets_flatten = [j for sublist in twitterlist for j in sublist]

    # Counting the number of key words in the tweets
    d = wordCount(tweets_flatten)
    
    # give a list of the sorted tweets, with most popular coming first
    d2 = sorted(d.items(), key = lambda x: x[1], reverse = True)
    d2_list.append(d2) 
    tw_final_all['Keywords'][i] = d2_list[i]
    d2_len.append(len(d2)) #a list of the lengths of the d2 list
   

        

In [563]:
#if no count keyword for that day, inserts (0,0)
for i in range(len(tw_date.index)):
    if len(tw_final_all['Keywords'][i])<max(d2_len):
        for j in range (max(d2_len)-len(tw_final_all['Keywords'][i])):
            tw_final_all['Keywords'][i].append((0,0))
        

In [564]:
#Spilits colom 'Keywords' into many coloums
for z in range(max(d2_len)):
    b=[]
    b.extend(a[z] for a in tw_final_all['Keywords'])
    o=z+1
    keyword=[]
    keyword.append('Keyword_'+str(o))
    keyword_str = ''.join(keyword)
    tw_final_all[keyword_str] = b

In [565]:
tw_final_all.head()

,date,Keywords,Keyword_1,Keyword_2,Keyword_3,Keyword_4,Keyword_5,Keyword_6,Keyword_7,Keyword_8,...,Keyword_21,Keyword_22,Keyword_23,Keyword_24,Keyword_25,Keyword_26,Keyword_27,Keyword_28,Keyword_29,Keyword_30
0,2016-12-14,"[(fog, 35), (hot, 33), (cold, 33), (sun, 32), ...","(fog, 35)","(hot, 33)","(cold, 33)","(sun, 32)","(warm, 17)","(ice, 12)","(sunny, 9)","(snow, 8)",...,"(heat, 1)","(0, 0)","(0, 0)","(0, 0)","(0, 0)","(0, 0)","(0, 0)","(0, 0)","(0, 0)","(0, 0)"
1,2016-12-15,"[(fog, 31), (hot, 29), (sun, 28), (cold, 28), ...","(fog, 31)","(hot, 29)","(sun, 28)","(cold, 28)","(warm, 16)","(sunny, 9)","(ice, 8)","(wind, 7)",...,"(0, 0)","(0, 0)","(0, 0)","(0, 0)","(0, 0)","(0, 0)","(0, 0)","(0, 0)","(0, 0)","(0, 0)"
2,2016-12-16,"[(hot, 74), (snow, 66), (fog, 61), (cold, 60),...","(hot, 74)","(snow, 66)","(fog, 61)","(cold, 60)","(sun, 60)","(warm, 48)","(storm, 44)","(ice, 33)",...,"(boiling, 4)","(frost, 3)","(visibility, 3)","(hail, 2)","(frosty, 1)","(icy, 1)","(snowing, 1)","(freezing, 1)","(foggy, 1)","(baltic, 1)"
3,2016-12-19,"[(hot, 74), (snow, 66), (fog, 61), (cold, 60),...","(hot, 74)","(snow, 66)","(fog, 61)","(cold, 60)","(sun, 60)","(warm, 46)","(storm, 44)","(ice, 33)",...,"(boiling, 4)","(frost, 3)","(visibility, 3)","(hail, 2)","(frosty, 1)","(icy, 1)","(snowing, 1)","(freezing, 1)","(foggy, 1)","(baltic, 1)"
4,2016-12-20,"[(hot, 76), (snow, 66), (cold, 63), (fog, 61),...","(hot, 76)","(snow, 66)","(cold, 63)","(fog, 61)","(sun, 60)","(warm, 49)","(storm, 44)","(ice, 33)",...,"(boiling, 4)","(frost, 3)","(visibility, 3)","(hail, 2)","(frosty, 1)","(icy, 1)","(snowing, 1)","(freezing, 1)","(foggy, 1)","(baltic, 1)"


In [566]:
del tw_final_all['Keywords'] #deletes col 'Keywords'

In [567]:
# Unpivots tw_final_all
tw_final2_all = pd.melt(tw_final_all, id_vars = ['date'])

In [568]:
tw_final2_all.head()

,date,variable,value
0,2016-12-14,Keyword_1,"(fog, 35)"
1,2016-12-15,Keyword_1,"(fog, 31)"
2,2016-12-16,Keyword_1,"(hot, 74)"
3,2016-12-19,Keyword_1,"(hot, 74)"
4,2016-12-20,Keyword_1,"(hot, 76)"


In [569]:
tw_final2_all.rename(columns = {'variable':'rank'}, inplace = True) #renames col

In [570]:
# Seperates tuples in value
b = []
c = []

b.extend(a[1] for a in tw_final2_all['value'])
tw_final2_all['count'] = b

c.extend(a[0] for a in tw_final2_all['value'])
tw_final2_all['value'] = c

In [571]:
tw_final2_all.head()

,date,rank,value,count
0,2016-12-14,Keyword_1,fog,35
1,2016-12-15,Keyword_1,fog,31
2,2016-12-16,Keyword_1,hot,74
3,2016-12-19,Keyword_1,hot,74
4,2016-12-20,Keyword_1,hot,76


In [572]:
tw_final2_all.rename(columns = {'value':'key_word'}, inplace = True)

In [573]:
tw_final2_all.head()

,date,rank,key_word,count
0,2016-12-14,Keyword_1,fog,35
1,2016-12-15,Keyword_1,fog,31
2,2016-12-16,Keyword_1,hot,74
3,2016-12-19,Keyword_1,hot,74
4,2016-12-20,Keyword_1,hot,76


In [574]:
#merged so that order of rank is in keyword order for each day
tw_final2_all = pd.merge(tw_final2_all, df_count, how='inner', on='date')

In [575]:
del tw_final2_all['total_count']

In [576]:
tw_final2_all.head()

,date,rank,key_word,count
0,2016-12-14,Keyword_1,fog,35
1,2016-12-14,Keyword_2,hot,33
2,2016-12-14,Keyword_3,cold,33
3,2016-12-14,Keyword_4,sun,32
4,2016-12-14,Keyword_5,warm,17


In [577]:
tw_final2_all=tw_final2_all.replace(0,np.nan)#replaces 0 with the NaN

In [578]:
tw_final2_all.to_csv(path_or_buf="C:\\Users\\Student24\\Documents\\twitter\\useing\\Daily_Word_Occurrence_all.csv", sep=',', na_rep='', \
               float_format=None, columns=['date', 'rank', 'key_word', 'count', 'total_count', '%daily word occurrence'], header=True, index=False, index_label=None, mode='w', encoding=None, \
               compression=None, quoting=None, quotechar='"', line_terminator='\n', chunksize=None, tupleize_cols=False, \
               date_format=None, doublequote=True, escapechar=None, decimal='.')
# Change header = false and mode = a from header = true and mode = w
#creates csv

In [579]:
tw_final2_all.head()

,date,rank,key_word,count
0,2016-12-14,Keyword_1,fog,35.0
1,2016-12-14,Keyword_2,hot,33.0
2,2016-12-14,Keyword_3,cold,33.0
3,2016-12-14,Keyword_4,sun,32.0
4,2016-12-14,Keyword_5,warm,17.0


# summer_winter plots

In [580]:
weather_words = ['breeze', 'breezy', 'cloudy', 'cold', 'ice', 'icy', 'icey', 'drizzle', 'frost', 'wind', 'mild', 'dew', 
                     'freezing', 'downpour', 'shower', 'rain', 'frost', 'nippy', 'hail', 'temperature', 'gail', 'gust',
                     'sleet', 'heat', 'storm', 'slush', 'fog', 'foggy', 'flood', 'visibility', 'warm', 'warmer',
                     'mist', 'frosty', 'misty', 'chilly', 'thunder', 'lightning', 'snow', 'snowing', 'hot', 'sun', 'sunny',
                     'boiling', 'baltic', 'burn']
#List of the weather words used for seach. Also above

In [581]:
weather_words_winter=['breeze', 'breezy', 'cloudy', 'cold', 'ice', 'icy', 'icey', 'drizzle', 'frost', 'wind', 'mild', 'dew', 
                     'freezing', 'downpour', 'shower', 'rain', 'frost', 'nippy', 'hail', 'temperature', 'gail', 'gust',
                     'sleet',  'storm', 'slush', 'fog', 'foggy', 'flood', 'visibility', 
                     'mist', 'frosty', 'misty', 'chilly', 'thunder', 'lightning', 'snow', 'snowing', 'baltic']
#Select words from weather_words that are winter words

In [582]:
weather_words_summer=['heat', 'warm', 'warmer','hot', 'sun', 'sunny','boiling',  'burn']
#Select words from weather_words that are summer words

###Plot of: Total count of Summer words vs Winter words over total time period

In [583]:
#To count total number of winter and summer words
count_winter=0
count_summer=0

for i in range (len(tw_final2_all)):
    if tw_final2_all['key_word'][i] in weather_words_winter:
        count_winter = count_winter + tw_final2_all['count'][i]
    elif tw_final2_all['key_word'][i] in weather_words_summer:
        count_summer = count_summer + tw_final2_all['count'][i]
        


In [584]:
count_winter, count_summer

(3478.0, 2473.0)

In [585]:
ratio_t=count_summer/count_winter
ratio_t= round(ratio_t, 2)
print ratio_t

0.71


In [586]:
trace = Bar(
    x=['Winter','Summer'], y=[count_winter,count_summer] 
    ,marker = dict(color=['blue','red'])
    )

data_3 = Data([trace])

layout_3 = go.Layout(
    title='Total count of Summer words vs Winter words over total time period. Summer to winter ratio='+str(ratio_t),
    yaxis = dict(title = 'Keywords Count')
) 

fig_3 = go.Figure(data=data_3, layout=layout_3)
py.plot(fig_3, filename = 'Summer_vs_winter_Totals') 
#py.plot(data, filename = 'Summer_vs_winter_Totals')

u'https://plot.ly/~ra123/12'

###Plots of: Occurrence and raito of Winter and Summer keywords per day 

In [587]:
tw_final2_all.head()

,date,rank,key_word,count
0,2016-12-14,Keyword_1,fog,35.0
1,2016-12-14,Keyword_2,hot,33.0
2,2016-12-14,Keyword_3,cold,33.0
3,2016-12-14,Keyword_4,sun,32.0
4,2016-12-14,Keyword_5,warm,17.0


In [588]:
#To get a count of winter and summer words for each day
count_winter_2_list=[]
count_summer_2_list=[]
i_list=[]
dump_list=[]

for j in range(len(df_count)):
    count_winter_2=0
    count_summer_2=0
    dump=0
    for i in range(max(d2_len)):
        i_list.append(i)
        x=len(i_list)-1 #to get an number of how many for loops have run
        if tw_final2_all['key_word'][x] in weather_words_winter:
                count_winter_2 = count_winter_2 + tw_final2_all['count'][x]
                #print tw_final2_all['count'][x]
        elif tw_final2_all['key_word'][x] in weather_words_summer:
                count_summer_2 = count_summer_2 + tw_final2_all['count'][x]
                #print tw_final2_all['count'][i]
        else:
            dump=dump + tw_final2_all['count'][x]
    
    count_winter_2_list.append(count_winter_2)
    count_summer_2_list.append(count_summer_2)
    dump_list.append(dump)
            
            
            

In [589]:
count_winter_2_list, count_summer_2_list, dump_list

([125.0,
  101.0,
  362.0,
  362.0,
  367.0,
  179.0,
  196.0,
  182.0,
  186.0,
  195.0,
  171.0,
  171.0,
  198.0,
  191.0,
  203.0,
  180.0,
  109.0],
 [98.0,
  89.0,
  211.0,
  209.0,
  214.0,
  134.0,
  149.0,
  140.0,
  144.0,
  148.0,
  127.0,
  128.0,
  150.0,
  148.0,
  154.0,
  138.0,
  92.0],
 [nan,
  nan,
  0,
  0,
  0,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan])

In [590]:
df_count_weather = DataFrame()
df_count_weather['date']=df_count['date']

In [591]:
df_count_weather

,date
0,2016-12-14
1,2016-12-15
2,2016-12-16
3,2016-12-19
4,2016-12-20
5,2016-12-21
6,2016-12-22
7,2016-12-23
8,2016-12-24
9,2016-12-25


In [592]:
df_count_weather['Winter']=count_winter_2_list
df_count_weather['Summer']=count_summer_2_list


In [593]:
df_count_weather['total_count']=df_count_weather['Winter']+df_count_weather['Summer']

In [594]:
df_count_weather['ratio_S/W']=df_count_weather['Summer'] / df_count_weather['Winter']

In [595]:
df_count_weather

,date,Winter,Summer,total_count,ratio_S/W
0,2016-12-14,125.0,98.0,223.0,0.784000
1,2016-12-15,101.0,89.0,190.0,0.881188
2,2016-12-16,362.0,211.0,573.0,0.582873
3,2016-12-19,362.0,209.0,571.0,0.577348
4,2016-12-20,367.0,214.0,581.0,0.583106
5,2016-12-21,179.0,134.0,313.0,0.748603
6,2016-12-22,196.0,149.0,345.0,0.760204
7,2016-12-23,182.0,140.0,322.0,0.769231
8,2016-12-24,186.0,144.0,330.0,0.774194
9,2016-12-25,195.0,148.0,343.0,0.758974


In [596]:
df_count_totals=df_count_weather[['date', 'total_count']]

In [597]:
df_count_totals

,date,total_count
0,2016-12-14,223.0
1,2016-12-15,190.0
2,2016-12-16,573.0
3,2016-12-19,571.0
4,2016-12-20,581.0
5,2016-12-21,313.0
6,2016-12-22,345.0
7,2016-12-23,322.0
8,2016-12-24,330.0
9,2016-12-25,343.0


In [598]:
df_count_weather_ws=df_count_weather[['date','Winter','Summer']]
# convert the df to a numpy array
np_counts_3 = np.array(df_count_weather_ws)
print np_counts_3.shape

(17L, 3L)


In [599]:
#code to plot count for keywords for each date. Code will update if any new keywords are added to table.
trace_3=[]
col_num_3 = len(df_count_weather_ws.columns) #Number of columns in dataframe to use in for loop
for i in range(col_num_3):
    if i==0:
        x_date_3=np_counts_3[:,i] # Sets the x-axis as the date
    else:
        y_count_3=np_counts_3[:,i].astype(float) #Sets y-axis as count
        trace_3.append(Scatter( x=x_date_3, y=y_count_3, name=df_count_weather_ws.dtypes.index[i] )) #appends list 'trace' with counts for each keyword
        
        
        
data_4 = Data(trace_3) #list trace is not data for plotly plot
layout_4 = go.Layout( #layout of plotly plot
    title='Occurrence of Winter and Summer keywords per day',
    xaxis=dict(
        title='Date',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Occurrence of Keywords',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )
)
fig_2 = go.Figure(data=data_4, layout=layout_4)
py.plot(fig_2, filename = 'winter_vs_summer_counts') #plotly plot

u'https://plot.ly/~ra123/14'

In [600]:
df_count_weather_r=df_count_weather[['date','ratio_S/W']]
# convert the df to a numpy array
np_counts_4 = np.array(df_count_weather_r)
print np_counts_4.shape

(17L, 2L)


In [601]:
#code to plot count for keywords for each date. Code will update if any new keywords are added to table.
trace_4=[]
col_num_4 = len(df_count_weather_r.columns) #Number of columns in dataframe to use in for loop
for i in range(col_num_4):
    if i==0:
        x_date_4=np_counts_4[:,i] # Sets the x-axis as the date
    else:
        y_count_4=np_counts_4[:,i].astype(float) #Sets y-axis as count
        trace_4.append(Scatter( x=x_date_4, y=y_count_4, name=df_count_weather_r.dtypes.index[i] )) #appends list 'trace' with counts for each keyword
        
        
        
data_5 = Data(trace_4) #list trace is not data for plotly plot
layout_5 = go.Layout( #layout of plotly plot
    title='Ratio of Summer to Winter keywords per day',
    xaxis=dict(
        title='Date',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Ratio of keywords (Summer/Winter)',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )
)
fig_2 = go.Figure(data=data_5, layout=layout_5)
py.plot(fig_2, filename = 'winter_vs_summer_ratio') #plotly plot

u'https://plot.ly/~ra123/16'

# Keyword_count_plots

In [602]:
 tw_final2_all.head()

,date,rank,key_word,count
0,2016-12-14,Keyword_1,fog,35.0
1,2016-12-14,Keyword_2,hot,33.0
2,2016-12-14,Keyword_3,cold,33.0
3,2016-12-14,Keyword_4,sun,32.0
4,2016-12-14,Keyword_5,warm,17.0


In [603]:
merged_all_sum=tw_final2_all[['key_word', 'count']]

In [604]:
df_wordcount_all=merged_all_sum.groupby('key_word').sum()
df_wordcount_all.reset_index(level = 0, inplace = True)
df_wordcount_all = df_wordcount_all.sort_values('count', axis=0, ascending=False, inplace=False, kind='quicksort', na_position='last')

In [605]:
df_wordcount_all.head()

,key_word,count
13,hot,879.0
6,fog,785.0
24,sun,754.0
4,cold,731.0
28,warm,518.0


In [606]:
trace = Bar(
    x= df_wordcount_all['key_word'], y=df_wordcount_all['count']
    )

data_5 = Data([trace])

layout_5 = go.Layout(
    title='Total count of all keywords',
    xaxis=dict(title='Keywords'),
    yaxis = dict(title = 'Keywords Count')
) 

fig_3 = go.Figure(data=data_5, layout=layout_5)
py.plot(fig_3, filename = 'Total_counts_of_all_word') 
#py.plot(data, filename = 'Summer_vs_winter_Totals')

u'https://plot.ly/~ra123/18'

In [607]:
merged_all_rank=tw_final2_all[['rank', 'key_word']]

In [608]:
merged_all_rank.head()

,rank,key_word
0,Keyword_1,fog
1,Keyword_2,hot
2,Keyword_3,cold
3,Keyword_4,sun
4,Keyword_5,warm


In [609]:
#inorder to get a dict key_words that apparear in each rank
i_list=[]

arraysDict = {}

for j in range (len(df_count)):
    for i in range (max(d2_len)):
        i_list.append(i)
        x=len(i_list)-1
        o=i+1
        if j==0:
            arraysDict['keyword{0}'.format(o)] = []
            arraysDict['keyword'+str(o)].append(merged_all_rank['key_word'][x])
            #print x
        else:
            arraysDict['keyword'+str(o)].append(merged_all_rank['key_word'][x])

      

In [610]:
#inorder to get a dict of the count of key_words that apparear in each rank
from collections import Counter
arraysDict_2 = {}

for i in range (len(arraysDict)):
    o=i+1
    counts_kw = Counter(arraysDict['keyword'+str(o)])
    arraysDict_2['{0}'.format(o)] = counts_kw


In [611]:
arraysDict_2

{'1': Counter({'fog': 3, 'hot': 14}),
 '10': Counter({'snow': 2, 'sunny': 9, 'temperature': 6}),
 '11': Counter({'lightning': 6, 'rain': 8, 'sunny': 3}),
 '12': Counter({'lightning': 8, 'rain': 9}),
 '13': Counter({'burn': 2, 'cloudy': 2, 'heat': 3, 'mild': 1, 'storm': 9}),
 '14': Counter({'boiling': 2,
          'burn': 1,
          'cloudy': 7,
          'lightning': 3,
          'mild': 2,
          'storm': 2}),
 '15': Counter({'boiling': 1, 'cloudy': 5, 'mild': 9, 'storm': 2}),
 '16': Counter({'burn': 3, 'mild': 2, 'shower': 11, 'storm': 1}),
 '17': Counter({'burn': 11, 'cloudy': 2, 'flood': 4}),
 '18': Counter({'baltic': 1,
          'boiling': 5,
          'flood': 2,
          'frost': 6,
          'shower': 3}),
 '19': Counter({'boiling': 6,
          'frost': 2,
          'heat': 1,
          'mild': 3,
          'mist': 3,
          'shower': 2}),
 '2': Counter({'fog': 8, 'hot': 2, 'snow': 3, 'sun': 4}),
 '20': Counter({nan: 1, 'baltic': 2, 'frost': 3, 'icy': 2, 'mist': 9}),

In [612]:
df_rank=pd.DataFrame.from_dict(arraysDict_2, orient='index') #dict to data frame
df_rank.reset_index(level = 0, inplace = True)
df_rank.rename(columns = {'index':'rank'}, inplace = True)


In [613]:
#so that we can order rank, now not strings so easyer
df_rank['rank']=df_rank['rank'].apply(int)

In [614]:
df_rank = df_rank.sort_values('rank', ascending=False)

In [615]:
df_rank.head()

,rank,nan,visibility,hail,baltic,frosty,icy,snowing,frost,mist,...,storm,ice,temperature,wind,rain,lightning,mild,burn,cloudy,shower
23,30,14.0,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,29,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,28,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,27,14.0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,26,14.0,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [616]:
np_rank = np.array(df_rank)
print np_rank.shape

(30L, 32L)


In [617]:
import colorlover as cl
from IPython.display import HTML

In [618]:
trace_rank=[]
col_num_rank = len(df_rank.columns) #Number of columns in dataframe to use in for loop
#N=3
for i in range(col_num_rank):
    if i==0:
        x_date_rank=np_rank[:,i] # Sets the x-axis as Rank
    elif i>1:
        y_count_rank=np_rank[:,i].astype(float) #Sets y-axis as count
        trace_rank.append(Bar( x=x_date_rank, y=y_count_rank, name=df_rank.dtypes.index[i])) #appends list 'trace' with counts for each keyword
        
        
        
data_rank = Data(trace_rank) #list trace is not data for plotly plot
layout_rank = go.Layout( #layout of plotly plot
    barmode='stack',
    title='The Occurrence of keywords in each rank. 1 being the highest',
    xaxis=dict(
        title='Rank',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
            #showticklabels=True
        )
    ),
    yaxis=dict(
        title='Occurrence of keywords',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )
)
fig_rank = go.Figure(data=data_rank, layout=layout_rank)
py.plot(fig_rank, filename = 'rank_of_keywords') #plotly plot

u'https://plot.ly/~ra123/22'

# Table for top 5key words

In [619]:
 tw_final2_all.head(10)

,date,rank,key_word,count
0,2016-12-14,Keyword_1,fog,35.0
1,2016-12-14,Keyword_2,hot,33.0
2,2016-12-14,Keyword_3,cold,33.0
3,2016-12-14,Keyword_4,sun,32.0
4,2016-12-14,Keyword_5,warm,17.0
5,2016-12-14,Keyword_6,ice,12.0
6,2016-12-14,Keyword_7,sunny,9.0
7,2016-12-14,Keyword_8,snow,8.0
8,2016-12-14,Keyword_9,wind,8.0
9,2016-12-14,Keyword_10,temperature,7.0


In [620]:
merged_top_five=merged_all[tw_final2_all['rank'].isin(['Keyword_1', 'Keyword_2', 'Keyword_3', 'Keyword_4', 'Keyword_5'])]
#to get top 5 keywords

In [621]:
merged_top_five=merged_top_five[['date','rank','key_word','count']]

In [622]:
# Joins dataframes tw_final and df_count on date
merged_top_five = pd.merge(merged_top_five, df_count_totals, how='inner', on='date')

In [623]:
merged_top_five['%daily word occurrence'] = (merged_top_five['count'] / merged_top_five['total_count']) * 100

In [624]:
merged_top_five.head()

,date,rank,key_word,count,total_count,%daily word occurrence
0,2016-12-14,Keyword_1,fog,35.0,223.0,15.695067
1,2016-12-14,Keyword_2,hot,33.0,223.0,14.798206
2,2016-12-14,Keyword_3,cold,33.0,223.0,14.798206
3,2016-12-14,Keyword_4,sun,32.0,223.0,14.349776
4,2016-12-14,Keyword_5,warm,17.0,223.0,7.623318


In [625]:
merged_top_five.to_csv(path_or_buf="C:\\Users\\Student24\\Documents\\twitter\\useing\\Daily_Word_Occurrence_top5.csv", sep=',', na_rep='', \
               float_format=None, columns=['date', 'rank', 'key_word', 'count', 'total_count', '%daily word occurrence'], header=True, index=False, index_label=None, mode='w', encoding=None, \
               compression=None, quoting=None, quotechar='"', line_terminator='\n', chunksize=None, tupleize_cols=False, \
               date_format=None, doublequote=True, escapechar=None, decimal='.')
# Change header = false and mode = a from header = true and mode = w

# count vs date plot for top 5 keywords

In [626]:
merged_piv=merged_top_five.pivot(index='date',columns='key_word', values='count') #piovts data based on date
merged_piv.reset_index(level = 0, inplace = True)
merged_piv


key_word,date,cold,fog,hot,snow,sun,warm
0,2016-12-14,33.0,35.0,33.0,NaN,32.0,17.0
1,2016-12-15,28.0,31.0,29.0,NaN,28.0,16.0
2,2016-12-16,60.0,61.0,74.0,66.0,60.0,NaN
3,2016-12-19,60.0,61.0,74.0,66.0,60.0,NaN
4,2016-12-20,63.0,61.0,76.0,66.0,60.0,NaN
5,2016-12-21,41.0,42.0,47.0,NaN,42.0,28.0
6,2016-12-22,43.0,49.0,55.0,NaN,45.0,32.0
7,2016-12-23,41.0,45.0,50.0,NaN,44.0,29.0
8,2016-12-24,42.0,46.0,53.0,NaN,44.0,30.0
9,2016-12-25,43.0,48.0,55.0,NaN,45.0,31.0


In [627]:
# convert the df to a numpy array
np_counts = np.array(merged_piv)
print np_counts.shape

(17L, 7L)


In [628]:
#code to plot count for keywords for each date. Code will update if any new keywords are added to table.
trace=[]
col_num = len(merged_piv.columns) #Number of columns in dataframe to use in for loop
for i in range(col_num):
    if i==0:
        x_date=np_counts[:,i] # Sets the x-axis as the date
    else:
        y_count=np_counts[:,i].astype(float) #Sets y-axis as count
        trace.append(Scatter( x=x_date, y=y_count, name=merged_piv.dtypes.index[i] )) #appends list 'trace' with counts for each keyword
        
        
        
data = Data(trace) #list trace is not data for plotly plot
layout = go.Layout( #layout of plotly plot
    title='Occurrence of keywords for top 5 keywords per day',
    xaxis=dict(
        title='Date',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Count',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )
)
fig = go.Figure(data=data, layout=layout)
py.plot(fig, filename = 'Occurrence_of_all_keywords') #plotly plot

u'https://plot.ly/~ra123/8'

# % vs date plot for Top 5 keywords

In [629]:
merged_piv_2=merged_top_five.pivot(index='date',columns='key_word', values='%daily word occurrence')
merged_piv_2.reset_index(level = 0, inplace = True)
merged_piv_2

key_word,date,cold,fog,hot,snow,sun,warm
0,2016-12-14,14.798206,15.695067,14.798206,NaN,14.349776,7.623318
1,2016-12-15,14.736842,16.315789,15.263158,NaN,14.736842,8.421053
2,2016-12-16,10.471204,10.645724,12.914485,11.518325,10.471204,NaN
3,2016-12-19,10.507881,10.683012,12.959720,11.558669,10.507881,NaN
4,2016-12-20,10.843373,10.499139,13.080895,11.359725,10.327022,NaN
5,2016-12-21,13.099042,13.418530,15.015974,NaN,13.418530,8.945687
6,2016-12-22,12.463768,14.202899,15.942029,NaN,13.043478,9.275362
7,2016-12-23,12.732919,13.975155,15.527950,NaN,13.664596,9.006211
8,2016-12-24,12.727273,13.939394,16.060606,NaN,13.333333,9.090909
9,2016-12-25,12.536443,13.994169,16.034985,NaN,13.119534,9.037901


In [630]:
# convert the df to a numpy array
np_counts_2 = np.array(merged_piv_2)
print np_counts_2.shape

(17L, 7L)


In [631]:
#code to plot count for keywords for each date. Code will update if any new keywords are added to table.
trace_2=[]
col_num_2 = len(merged_piv_2.columns) #Number of columns in dataframe to use in for loop
for i in range(col_num_2):
    if i==0:
        x_date_2=np_counts_2[:,i] # Sets the x-axis as the date
    else:
        y_count_2=np_counts_2[:,i].astype(float) #Sets y-axis as count
        trace_2.append(Scatter( x=x_date_2, y=y_count_2, name=merged_piv_2.dtypes.index[i] )) #appends list 'trace' with counts for each keyword
        
        
        
data_2 = Data(trace_2) #list trace is not data for plotly plot
layout_2 = go.Layout( #layout of plotly plot
    title='% of keywords for top 5 keywords per day',
    xaxis=dict(
        title='Date',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='%',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )
)
fig_2 = go.Figure(data=data_2, layout=layout_2)
py.plot(fig_2, filename = '%_of_all_keywords') #plotly plot

u'https://plot.ly/~ra123/10'